In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install hdbscan

In [2]:
import numpy as np
# import sys
import copy
import hdbscan

In [ ]:
a = 19

if a < 10:
  input_file = f"/content/drive/MyDrive/Kurtzman/clusters/cluster.00000{a}.pdb"
elif a<100:
  input_file = f"/content/drive/MyDrive/Kurtzman/clusters/cluster.0000{a}.pdb"
elif a<1000:
  input_file = f"/content/drive/MyDrive/Kurtzman/clusters/cluster.000{a}.pdb"

with open(input_file, "r") as file:  # opens the file and reads each line as an item in a list and closes it.

  np.seterr(divide='ignore', invalid='ignore')
  in_file = file.readlines()  # converts what is read to a list
  end = len(in_file)

  oxygen = in_file[1:end:3]  # extracting oxygen from in_file
  h1 = in_file[2:end:3]  # extracting h1 from in_file
  h2 = in_file[3:end:3]  # extracting h2 from in_file
  ###########################################################################

# Setting new coordinates for O for each water molecule.
oxygen_x = []
for i in oxygen:
  oxygen_x.append(float(i[32:38]))
np_o_x = np.array(oxygen_x)
new_oxygen_x = np.zeros_like(np_o_x)

oxygen_y = []
for j in oxygen:
  oxygen_y.append(float(j[40:46]))
np_o_y = np.array(oxygen_y)
new_oxygen_y = np.zeros_like(np_o_y)

oxygen_z = []
for k in oxygen:
  oxygen_z.append(float(k[48:54]))
np_o_z = np.array(oxygen_z)
new_oxygen_z = np.zeros_like(np_o_z)
#############################################################################

# Setting new coordinates for h1 with respect to O atom for each water molecule
h1_x = []
for i in h1:
  h1_x.append(float(i[32:38]))
np_h1_x = np.array(h1_x)
new_h1_x = np.subtract(np_h1_x, np_o_x)

h1_y = []
for j in h1:
  h1_y.append(float(j[40:46]))
np_h1_y = np.array(h1_y)
new_h1_y = np.subtract(np_h1_y, np_o_y)

h1_z = []
for k in h1:
  h1_z.append(float(k[48:54]))
np_h1_z = np.array(h1_z)
new_h1_z = np.subtract(np_h1_z, np_o_z)
#############################################################################

# Setting new coordinates for h2 with respect to O atom for each water molecule.
h2_x = []
for i in h2:
  h2_x.append(float(i[32:38]))
np_h2_x = np.array(h2_x)
new_h2_x = np.subtract(np_h2_x, np_o_x)


h2_y = []
for j in h2:
  h2_y.append(float(j[40:46]))
np_h2_y = np.array(h2_y)
new_h2_y = np.subtract(np_h2_y, np_o_y)

h2_z = []
for k in h2:
  h2_z.append(float(k[48:54]))
np_h2_z = np.array(h2_z)
new_h2_z = np.subtract(np_h2_z, np_o_z)

#############################################################################
center_o_x_min = np.min(np_o_x)
center_o_x_max = np.max(np_o_x)
center_o_y_min = np.min(np_o_y)
center_o_y_max = np.max(np_o_y)
center_o_z_min = np.min(np_o_z)
center_o_z_max = np.max(np_o_z)
cluster_o_center_x = ((center_o_x_max+center_o_x_min)/2)
cluster_o_center_y = ((center_o_y_max+center_o_y_min)/2)
cluster_o_center_z = ((center_o_z_max+center_o_z_min)/2)

diff_cluster_individual_x = np.subtract(np_o_x, cluster_o_center_x)
diff_cluster_individual_y = np.subtract(np_o_y, cluster_o_center_y)
diff_cluster_individual_z = np.subtract(np_o_z, cluster_o_center_z)

#############################################################################

#  Getting the length of h1 vector for each water molecule.
np.set_printoptions(suppress=True,
                  formatter={'float_kind': '{:0.8f}'.format})

h1_x_squares = np.square(new_h1_x)
h1_y_squares = np.square(new_h1_y)
h1_z_squares = np.square(new_h1_z)
h1_xy_squares_added = np.add(np.square(new_h1_x), np.square(new_h1_y))
h1_all_squares_added = np.add(h1_xy_squares_added, np.square(new_h1_z))
h1_length = np.sqrt(h1_all_squares_added)

#  Getting the length of h2 vector for each water molecule.
np.set_printoptions(suppress=True,
                  formatter={'float_kind': '{:0.8f}'.format})
h2_x_squares = np.square(new_h2_x)
h2_y_squares = np.square(new_h2_y)
h2_z_squares = np.square(new_h2_z)
h2_xy_squares_added = np.add(h2_x_squares, h2_y_squares)
h2_all_squares_added = np.add(h2_xy_squares_added, h2_z_squares)
h2_length = np.sqrt(h2_all_squares_added)

# #############################################################################

#  Normalizing each axis of each hydrogen atom. we do this by dividing each atom axes with its own atom magnitude from oxygen atom

norm_h1_x = np.divide(new_h1_x, h1_length)
norm_h1_y = np.divide(new_h1_y, h1_length)
norm_h1_z = np.divide(new_h1_z, h1_length)
norm_h2_x = np.divide(new_h2_x, h2_length)
norm_h2_y = np.divide(new_h2_y, h2_length)
norm_h2_z = np.divide(new_h2_z, h2_length)

x_ref = [1, 0, 0]
y_ref = [0, 1, 0]
z_ref = [0, 0, 1]
#############################################################################

temp_norm_h1_x = copy.deepcopy(norm_h1_x)
temp_norm_h1_y = copy.deepcopy(norm_h1_y)
temp_norm_h1_z = copy.deepcopy(norm_h1_z)
temp_norm_h2_x = copy.deepcopy(norm_h2_x)
temp_norm_h2_y = copy.deepcopy(norm_h2_y)
temp_norm_h2_z = copy.deepcopy(norm_h2_z)

H1final_q_0_all = []
H1final_q_1_all = []
H1final_q_2_all = []
H1final_q_3_all = []
H2final_q_0_all = []
H2final_q_1_all = []
H2final_q_2_all = []
H2final_q_3_all = []

H1updated_theta_in_degrees_all = []
H2updated_theta_in_degrees_all = []
H1updated_theta3p_in_degrees_all = []
H2updated_theta3p_in_degrees_all = []


for ndx, i in enumerate(norm_h1_x):

  H1ar_x = np.subtract(np.multiply(temp_norm_h1_y[ndx], x_ref[2]), np.multiply(temp_norm_h1_z[ndx], x_ref[1]))
  H1ar_y = np.subtract(np.multiply(temp_norm_h1_z[ndx], x_ref[0]), np.multiply(temp_norm_h1_x[ndx], x_ref[2]))
  H1ar_z = np.subtract(np.multiply(temp_norm_h1_x[ndx], x_ref[1]), np.multiply(temp_norm_h1_y[ndx], x_ref[0]))

  #  Normalizing the first rotational axis (ar).

  H1ar_x_squares = np.square(H1ar_x)
  H1ar_y_squares = np.square(H1ar_y)
  H1ar_z_squares = np.square(H1ar_z)
  H1ar_xy_squares_added = np.add(H1ar_x_squares, H1ar_y_squares)
  H1ar_all_squares_added = np.add(H1ar_xy_squares_added, H1ar_z_squares)
  H1ar_length = np.sqrt(H1ar_all_squares_added)

  H1norm_ar_x = np.nan_to_num(np.divide(H1ar_x, H1ar_length))
  H1norm_ar_y = np.nan_to_num(np.divide(H1ar_y, H1ar_length))
  H1norm_ar_z = np.nan_to_num(np.divide(H1ar_z, H1ar_length))
  # ###########################################################################

  # Calculating theta from the dot product of x_ref vector and h1 vector

  H1dotproduct_h1_xref_xcomponent = np.multiply(temp_norm_h1_x[ndx], x_ref[0])
  H1dotproduct_h1_xref_ycomponent = np.multiply(temp_norm_h1_y[ndx], x_ref[1])  # I know it's zero and that's because x_ref is [1.0.0]
  H1dotproduct_h1_xref_zcomponent = np.multiply(temp_norm_h1_z[ndx], x_ref[2])  # I know it's zero and that's because x_ref is [1.0.0]
  H1dotproduct_h1_xref_xycomponents = np.add(H1dotproduct_h1_xref_xcomponent, H1dotproduct_h1_xref_ycomponent)
  H1dotproduct_h1_xref_allcomponents = np.add(H1dotproduct_h1_xref_xycomponents, H1dotproduct_h1_xref_zcomponent)
  H1theta_in_radians = np.arccos(H1dotproduct_h1_xref_allcomponents)
  H1theta_in_degrees = np.degrees(H1theta_in_radians)

  # ###########################################################################
  #  Test to check for theta sign.

  H1dotproduct_ar_normh1_xcomponent = np.multiply(H1ar_x, temp_norm_h1_x[ndx])
  H1dotproduct_ar_normh1_ycomponent = np.multiply(H1ar_y, temp_norm_h1_y[ndx])
  H1dotproduct_ar_normh1_zcomponent = np.multiply(H1ar_z, temp_norm_h1_z[ndx])
  H1dotproduct_ar_normh1_xycomponents = np.add(H1dotproduct_ar_normh1_xcomponent, H1dotproduct_ar_normh1_ycomponent)
  H1dotproduct_ar_normh1_allcomponents = np.add(H1dotproduct_ar_normh1_xycomponents, H1dotproduct_ar_normh1_zcomponent)

  H1updated_theta_in_radians = np.zeros_like(H1theta_in_radians)

  if H1dotproduct_ar_normh1_allcomponents > 0:
      H1updated_theta_in_radians = H1theta_in_radians / 2.0
  else:
      H1updated_theta_in_radians = H1theta_in_radians / -2.0

  H1updated_theta_in_degrees = np.degrees(H1updated_theta_in_radians)
  H1updated_theta_in_degrees_all.append(H1updated_theta_in_degrees.tolist())

  # ###########################################################################

  #  Defining first quaternion based on the first axis of rotation (ar) (H1q1 0,1,2,3)

  H1q1_0 = np.cos(H1updated_theta_in_radians)
  H1q1_1 = np.multiply(H1norm_ar_x, np.sin(H1updated_theta_in_radians))
  H1q1_2 = np.multiply(H1norm_ar_y, np.sin(H1updated_theta_in_radians))
  H1q1_3 = np.multiply(H1norm_ar_z, np.sin(H1updated_theta_in_radians))

  e_0 = copy.deepcopy(H1q1_0)
  e_1 = copy.deepcopy(H1q1_1)
  e_2 = copy.deepcopy(H1q1_2)
  e_3 = copy.deepcopy(H1q1_3)

  # ###########################################################################

  #getting h1_moved_x,y,z:

  htemp_0 = np.multiply(np.subtract(np.add(np.square(e_0), np.square(e_1)), np.add(np.square(e_2), np.square(e_3))), temp_norm_h1_x[ndx])
  htemp_0 += np.multiply(2, np.multiply(np.add(np.multiply(e_1, e_2), np.multiply(e_0, e_3)), temp_norm_h1_y[ndx]))
  htemp_0 += np.multiply(2, np.multiply(np.subtract(np.multiply(e_1, e_3), np.multiply(e_0, e_2)), temp_norm_h1_z[ndx]))

  htemp_1 = np.multiply(np.multiply(2, np.subtract(np.multiply(e_1, e_2), np.multiply(e_0, e_3))), temp_norm_h1_x[ndx])
  htemp_1 += np.multiply(np.add(np.subtract(np.multiply(e_0, e_0), np.multiply(e_1, e_1)), np.subtract(np.multiply(e_2, e_2), np.multiply(e_3, e_3))), temp_norm_h1_y[ndx])
  htemp_1 += np.multiply(2, np.multiply(np.add(np.multiply(e_2, e_3), np.multiply(e_0, e_1)), temp_norm_h1_z[ndx]))

  htemp_2 = np.multiply(np.multiply(2, np.add(np.multiply(e_1, e_3), np.multiply(e_0, e_2))), temp_norm_h1_x[ndx])
  htemp_2 += np.multiply(2, np.multiply(np.subtract(np.multiply(e_2, e_3), np.multiply(e_0, e_1)), temp_norm_h1_y[ndx]))
  htemp_2 += np.multiply(np.add(np.subtract(np.subtract(np.multiply(e_0, e_0), np.multiply(e_1, e_1)), np.multiply(e_2, e_2)), np.multiply(e_3, e_3)), temp_norm_h1_z[ndx])

  h1_moved_x = copy.deepcopy(htemp_0)
  h1_moved_y = copy.deepcopy(htemp_1)
  h1_moved_z = copy.deepcopy(htemp_2)

  ###########################################################################

  #getting h2_after_h1_moved_x,y,z:

  htemp_0 = np.multiply(np.subtract(np.add(np.square(e_0), np.square(e_1)), np.add(np.square(e_2), np.square(e_3))), temp_norm_h2_x[ndx])
  htemp_0 += np.multiply(2, np.multiply(np.add(np.multiply(e_1, e_2), np.multiply(e_0, e_3)), temp_norm_h2_y[ndx]))
  htemp_0 += np.multiply(2, np.multiply(np.subtract(np.multiply(e_1, e_3), np.multiply(e_0, e_2)), temp_norm_h2_z[ndx]))

  htemp_1 = np.multiply(np.multiply(2, np.subtract(np.multiply(e_1, e_2), np.multiply(e_0, e_3))), temp_norm_h2_x[ndx])
  htemp_1 += np.multiply(np.add(np.subtract(np.multiply(e_0, e_0), np.multiply(e_1, e_1)), np.subtract(np.multiply(e_2, e_2), np.multiply(e_3, e_3))), temp_norm_h2_y[ndx])
  htemp_1 += np.multiply(2, np.multiply(np.add(np.multiply(e_2, e_3), np.multiply(e_0, e_1)), temp_norm_h2_z[ndx]))

  htemp_2 = np.multiply(np.multiply(2, np.add(np.multiply(e_1, e_3), np.multiply(e_0, e_2))), temp_norm_h2_x[ndx])
  htemp_2 += np.multiply(2, np.multiply(np.subtract(np.multiply(e_2, e_3), np.multiply(e_0, e_1)), temp_norm_h2_y[ndx]))
  htemp_2 += np.multiply(np.add(np.subtract(np.subtract(np.multiply(e_0, e_0), np.multiply(e_1, e_1)), np.multiply(e_2, e_2)), np.multiply(e_3, e_3)), temp_norm_h2_z[ndx])

  h2_after_h1_moved_x = copy.deepcopy(htemp_0)
  h2_after_h1_moved_y = copy.deepcopy(htemp_1)
  h2_after_h1_moved_z = copy.deepcopy(htemp_2)
  # ###########################################################################

  H1z_mol_vect_x = np.subtract(np.multiply(h1_moved_y, h2_after_h1_moved_z), np.multiply(h1_moved_z, h2_after_h1_moved_y))
  H1z_mol_vect_y = np.subtract(np.multiply(h1_moved_z, h2_after_h1_moved_x), np.multiply(h1_moved_x, h2_after_h1_moved_z))
  H1z_mol_vect_z = np.subtract(np.multiply(h1_moved_x, h2_after_h1_moved_y), np.multiply(h1_moved_y, h2_after_h1_moved_x))

  H1z_mol_vect_x_squares = np.square(H1z_mol_vect_x)
  H1z_mol_vect_y_squares = np.square(H1z_mol_vect_y)
  H1z_mol_vect_z_squares = np.square(H1z_mol_vect_z)
  H1z_mol_vect_xy_squares_added = np.add(H1z_mol_vect_x_squares, H1z_mol_vect_y_squares)
  H1z_mol_vect_all_squares_added = np.add(H1z_mol_vect_xy_squares_added, H1z_mol_vect_z_squares)
  H1z_mol_vect_length = np.sqrt(H1z_mol_vect_all_squares_added)

  H1norm_z_mol_vect_x = np.divide(H1z_mol_vect_x, H1z_mol_vect_length)
  H1norm_z_mol_vect_y = np.divide(H1z_mol_vect_y, H1z_mol_vect_length)
  H1norm_z_mol_vect_z = np.divide(H1z_mol_vect_z, H1z_mol_vect_length)

  ###########################################################################

  H1dotproduct_norm_z_mol_vect_zref_xcomponent = np.multiply(H1norm_z_mol_vect_x, z_ref[0])  # I know it's zero and that's because z_ref is [0,0,1]
  H1dotproduct_norm_z_mol_vect_zref_ycomponent = np.multiply(H1norm_z_mol_vect_y, z_ref[1])  # I know it's zero and that's because z_ref is [0,0,1]
  H1dotproduct_norm_z_mol_vect_zref_zcomponent = np.multiply(H1norm_z_mol_vect_z, z_ref[2])
  H1dotproduct_norm_z_mol_vect_zref_xycomponents = np.add(H1dotproduct_norm_z_mol_vect_zref_xcomponent, H1dotproduct_norm_z_mol_vect_zref_ycomponent)
  H1dotproduct_norm_z_mol_vect_zref_allcomponents = np.add(H1dotproduct_norm_z_mol_vect_zref_xycomponents, H1dotproduct_norm_z_mol_vect_zref_zcomponent)

  H1theta3p_in_radians = np.arccos(H1dotproduct_norm_z_mol_vect_zref_allcomponents)
  H1theta3p_in_degrees = np.degrees(H1theta3p_in_radians)

  H1crossp_norm_z_mol_vect_z_ref_x = np.subtract(np.multiply(H1norm_z_mol_vect_y, z_ref[2]), np.multiply(H1norm_z_mol_vect_z, z_ref[1]))
  H1crossp_norm_z_mol_vect_z_ref_y = np.subtract(np.multiply(H1norm_z_mol_vect_z, z_ref[0]), np.multiply(H1norm_z_mol_vect_x, z_ref[2]))
  H1crossp_norm_z_mol_vect_z_ref_z = np.subtract(np.multiply(H1norm_z_mol_vect_x, z_ref[1]), np.multiply(H1norm_z_mol_vect_y, z_ref[0]))

  H1dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_xcomponent = np.multiply(H1crossp_norm_z_mol_vect_z_ref_x, h1_moved_x)
  H1dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_ycomponent = np.multiply(H1crossp_norm_z_mol_vect_z_ref_y, h1_moved_y)
  H1dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_zcomponent = np.multiply(H1crossp_norm_z_mol_vect_z_ref_z, h1_moved_z)
  H1dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_xycomponents = np.add(H1dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_xcomponent, H1dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_ycomponent)
  H1dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_allcomponents = np.add(H1dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_xycomponents, H1dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_zcomponent)

  H1updated_theta3p_in_radians = np.zeros_like(H1theta3p_in_radians)

  if H1dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_allcomponents < 0.0:
      H1updated_theta3p_in_radians = H1theta3p_in_radians / 2.0
  else:
      H1updated_theta3p_in_radians = H1theta3p_in_radians / -2.0


  H1updated_theta3p_in_degrees = np.degrees(H1updated_theta3p_in_radians)
  H1updated_theta3p_in_degrees_all.append(H1updated_theta3p_in_degrees.tolist())
  # ###########################################################################

  #  Defining second quaternion based on axis of rotation and angle. (h1q2 0,1,2,3)

  H1q2_0 = np.cos(H1updated_theta3p_in_radians)
  H1q2_1 = np.multiply(x_ref[0], np.sin(H1updated_theta3p_in_radians))
  H1q2_2 = np.multiply(x_ref[1], np.sin(H1updated_theta3p_in_radians))
  H1q2_3 = np.multiply(x_ref[2], np.sin(H1updated_theta3p_in_radians))
  # ###########################################################################

  #  Calculating the product quaternion. (H1q1 . H1q2)


  e_0 = np.subtract(np.subtract(np.subtract(np.multiply(H1q1_0, H1q2_0), np.multiply(H1q1_1, H1q2_1)), np.multiply(H1q1_2, H1q2_2)), np.multiply(H1q1_3, H1q2_3))
  e_1 = np.subtract(np.add(np.add(np.multiply(H1q1_0, H1q2_1), np.multiply(H1q1_1, H1q2_0)), np.multiply(H1q1_2, H1q2_3)), np.multiply(H1q1_3, H1q2_2))
  e_2 = np.add(np.add(np.subtract(np.multiply(H1q1_0, H1q2_2), np.multiply(H1q1_1, H1q2_3)), np.multiply(H1q1_2, H1q2_0)), np.multiply(H1q1_3, H1q2_1))
  e_3 = np.add(np.subtract(np.add(np.multiply(H1q1_0, H1q2_3), np.multiply(H1q1_1, H1q2_2)), np.multiply(H1q1_2, H1q2_1)), np.multiply(H1q1_3, H1q2_0))
  ##########################################################################

  H1final_q_0 = copy.deepcopy(e_0)
  H1final_q_1 = copy.deepcopy(e_1)
  H1final_q_2 = copy.deepcopy(e_2)
  H1final_q_3 = copy.deepcopy(e_3)

  # H1q1 all
  H1final_q_0_all.append(H1final_q_0)
  H1final_q_1_all.append(H1final_q_1)
  H1final_q_2_all.append(H1final_q_2)
  H1final_q_3_all.append(H1final_q_3)

  # Transformation of H2
  H2ar_x = np.subtract(np.multiply(temp_norm_h2_y[ndx], x_ref[2]), np.multiply(temp_norm_h2_z[ndx], x_ref[1]))
  H2ar_y = np.subtract(np.multiply(temp_norm_h2_z[ndx], x_ref[0]), np.multiply(temp_norm_h2_x[ndx], x_ref[2]))
  H2ar_z = np.subtract(np.multiply(temp_norm_h2_x[ndx], x_ref[1]), np.multiply(temp_norm_h2_y[ndx], x_ref[0]))

  #  Normalizing the first rotational axis (ar) for h2.

  H2ar_x_squares = np.square(H2ar_x)
  H2ar_y_squares = np.square(H2ar_y)
  H2ar_z_squares = np.square(H2ar_z)
  H2ar_xy_squares_added = np.add(H2ar_x_squares, H2ar_y_squares)
  H2ar_all_squares_added = np.add(H2ar_xy_squares_added, H2ar_z_squares)
  H2ar_length = np.sqrt(H2ar_all_squares_added)

  H2norm_ar_x = np.nan_to_num(np.divide(H2ar_x, H2ar_length))
  H2norm_ar_y = np.nan_to_num(np.divide(H2ar_y, H2ar_length))
  H2norm_ar_z = np.nan_to_num(np.divide(H2ar_z, H2ar_length))

  # ###########################################################################

  #  Calculating theta from the dot product of x_ref vector and h2 vector

  H2dotproduct_h2_xref_xcomponent = np.multiply(temp_norm_h2_x[ndx], x_ref[0])
  H2dotproduct_h2_xref_ycomponent = np.multiply(temp_norm_h2_y[ndx], x_ref[1])  # I know it's zero and that's because x_ref is [1.0.0]
  H2dotproduct_h2_xref_zcomponent = np.multiply(temp_norm_h2_z[ndx], x_ref[2])  # I know it's zero and that's because x_ref is [1.0.0]
  H2dotproduct_h2_xref_xycomponents = np.add(H2dotproduct_h2_xref_xcomponent, H2dotproduct_h2_xref_ycomponent)
  H2dotproduct_h2_xref_allcomponents = np.add(H2dotproduct_h2_xref_xycomponents, H2dotproduct_h2_xref_zcomponent)
  H2theta_in_radians = np.arccos(H2dotproduct_h2_xref_allcomponents)
  H2theta_in_degrees = np.degrees(H2theta_in_radians)
  ###########################################################################

  #  Test to check for theta sign.
  H2dotproduct_ar_normh2_xcomponent = np.multiply(H2ar_x, temp_norm_h2_x[ndx])
  H2dotproduct_ar_normh2_ycomponent = np.multiply(H2ar_y, temp_norm_h2_y[ndx])
  H2dotproduct_ar_normh2_zcomponent = np.multiply(H2ar_z, temp_norm_h2_z[ndx])
  H2dotproduct_ar_normh2_xycomponents = np.add(H2dotproduct_ar_normh2_xcomponent, H2dotproduct_ar_normh2_ycomponent)
  H2dotproduct_ar_normh2_allcomponents = np.add(H2dotproduct_ar_normh2_xycomponents, H2dotproduct_ar_normh2_zcomponent)

  H2updated_theta_in_radians = np.zeros_like(H2theta_in_radians)

  if H2dotproduct_ar_normh2_allcomponents > 0:
      H2updated_theta_in_radians = H2theta_in_radians / 2.0
  else:
      H2updated_theta_in_radians = H2theta_in_radians / -2.0

  H2updated_theta_in_degrees = np.degrees(H2updated_theta_in_radians)
  H2updated_theta_in_degrees_all.append(H2updated_theta_in_degrees.tolist())

  # ###############################################################################

  #  Defining first quaternion based on the first axis of rotation (ar) for h2 (H2q1 0,1,2,3)

  H2q1_0 = np.cos(H2updated_theta_in_radians)
  H2q1_1 = np.multiply(H2norm_ar_x, np.sin(H2updated_theta_in_radians))
  H2q1_2 = np.multiply(H2norm_ar_y, np.sin(H2updated_theta_in_radians))
  H2q1_3 = np.multiply(H2norm_ar_z, np.sin(H2updated_theta_in_radians))

  e_0 = copy.deepcopy(H2q1_0)
  e_1 = copy.deepcopy(H2q1_1)
  e_2 = copy.deepcopy(H2q1_2)
  e_3 = copy.deepcopy(H2q1_3)
  ###############################################################################

  htemp_0 = np.multiply(np.subtract(np.add(np.square(e_0), np.square(e_1)), np.add(np.square(e_2), np.square(e_3))), temp_norm_h2_x[ndx])
  htemp_0 += np.multiply(2, np.multiply(np.add(np.multiply(e_1, e_2), np.multiply(e_0, e_3)), temp_norm_h2_y[ndx]))
  htemp_0 += np.multiply(2, np.multiply(np.subtract(np.multiply(e_1, e_3), np.multiply(e_0, e_2)), temp_norm_h2_z[ndx]))

  htemp_1 = np.multiply(np.multiply(2, np.subtract(np.multiply(e_1, e_2), np.multiply(e_0, e_3))), temp_norm_h2_x[ndx])
  htemp_1 += np.multiply(np.add(np.subtract(np.multiply(e_0, e_0), np.multiply(e_1, e_1)), np.subtract(np.multiply(e_2, e_2), np.multiply(e_3, e_3))), temp_norm_h2_y[ndx])
  htemp_1 += np.multiply(2, np.multiply(np.add(np.multiply(e_2, e_3), np.multiply(e_0, e_1)), temp_norm_h2_z[ndx]))

  htemp_2 = np.multiply(np.multiply(2, np.add(np.multiply(e_1, e_3), np.multiply(e_0, e_2))), temp_norm_h2_x[ndx])
  htemp_2 += np.multiply(2, np.multiply(np.subtract(np.multiply(e_2, e_3), np.multiply(e_0, e_1)), temp_norm_h2_y[ndx]))
  htemp_2 += np.multiply(np.add(np.subtract(np.subtract(np.multiply(e_0, e_0), np.multiply(e_1, e_1)), np.multiply(e_2, e_2)), np.multiply(e_3, e_3)), temp_norm_h2_z[ndx])

  h2_moved_x = copy.deepcopy(htemp_0)
  h2_moved_y = copy.deepcopy(htemp_1)
  h2_moved_z = copy.deepcopy(htemp_2)
  ###############################################################################

  htemp_0 = np.multiply(np.subtract(np.add(np.square(e_0), np.square(e_1)), np.add(np.square(e_2), np.square(e_3))), temp_norm_h1_x[ndx])
  htemp_0 += np.multiply(2, np.multiply(np.add(np.multiply(e_1, e_2), np.multiply(e_0, e_3)), temp_norm_h1_y[ndx]))
  htemp_0 += np.multiply(2, np.multiply(np.subtract(np.multiply(e_1, e_3), np.multiply(e_0, e_2)), temp_norm_h1_z[ndx]))

  htemp_1 = np.multiply(np.multiply(2, np.subtract(np.multiply(e_1, e_2), np.multiply(e_0, e_3))), temp_norm_h1_x[ndx])
  htemp_1 += np.multiply(np.add(np.subtract(np.multiply(e_0, e_0), np.multiply(e_1, e_1)), np.subtract(np.multiply(e_2, e_2), np.multiply(e_3, e_3))), temp_norm_h1_y[ndx])
  htemp_1 += np.multiply(2, np.multiply(np.add(np.multiply(e_2, e_3), np.multiply(e_0, e_1)), temp_norm_h1_z[ndx]))

  htemp_2 = np.multiply(np.multiply(2, np.add(np.multiply(e_1, e_3), np.multiply(e_0, e_2))), temp_norm_h1_x[ndx])
  htemp_2 += np.multiply(2, np.multiply(np.subtract(np.multiply(e_2, e_3), np.multiply(e_0, e_1)), temp_norm_h1_y[ndx]))
  htemp_2 += np.multiply(np.add(np.subtract(np.subtract(np.multiply(e_0, e_0), np.multiply(e_1, e_1)), np.multiply(e_2, e_2)), np.multiply(e_3, e_3)), temp_norm_h1_z[ndx])

  h1_after_h2_moved_x = copy.deepcopy(htemp_0)
  h1_after_h2_moved_y = copy.deepcopy(htemp_1)
  h1_after_h2_moved_z = copy.deepcopy(htemp_2)
  ###############################################################################

  H2z_mol_vect_x = np.subtract(np.multiply(h2_moved_y, h1_after_h2_moved_z), np.multiply(h2_moved_z, h1_after_h2_moved_y))
  H2z_mol_vect_y = np.subtract(np.multiply(h2_moved_z, h1_after_h2_moved_x), np.multiply(h2_moved_x, h1_after_h2_moved_z))
  H2z_mol_vect_z = np.subtract(np.multiply(h2_moved_x, h1_after_h2_moved_y), np.multiply(h2_moved_y, h1_after_h2_moved_x))

  H2z_mol_vect_x_squares = np.square(H2z_mol_vect_x)
  H2z_mol_vect_y_squares = np.square(H2z_mol_vect_y)
  H2z_mol_vect_z_squares = np.square(H2z_mol_vect_z)
  H2z_mol_vect_xy_squares_added = np.add(H2z_mol_vect_x_squares, H2z_mol_vect_y_squares)
  H2z_mol_vect_all_squares_added = np.add(H2z_mol_vect_xy_squares_added, H2z_mol_vect_z_squares)
  H2z_mol_vect_length = np.sqrt(H2z_mol_vect_all_squares_added)

  H2norm_z_mol_vect_x = np.divide(H2z_mol_vect_x, H2z_mol_vect_length)
  H2norm_z_mol_vect_y = np.divide(H2z_mol_vect_y, H2z_mol_vect_length)
  H2norm_z_mol_vect_z = np.divide(H2z_mol_vect_z, H2z_mol_vect_length)
  ###############################################################################

  H2dotproduct_norm_z_mol_vect_zref_xcomponent = np.multiply(H2norm_z_mol_vect_x, z_ref[0])
  H2dotproduct_norm_z_mol_vect_zref_ycomponent = np.multiply(H2norm_z_mol_vect_y, z_ref[1])
  H2dotproduct_norm_z_mol_vect_zref_zcomponent = np.multiply(H2norm_z_mol_vect_z, z_ref[2])
  H2dotproduct_norm_z_mol_vect_zref_xycomponents = np.add(H2dotproduct_norm_z_mol_vect_zref_xcomponent, H2dotproduct_norm_z_mol_vect_zref_ycomponent)
  H2dotproduct_norm_z_mol_vect_zref_allcomponents = np.add(H2dotproduct_norm_z_mol_vect_zref_xycomponents, H2dotproduct_norm_z_mol_vect_zref_zcomponent)

  H2theta3p_in_radians = np.arccos(H2dotproduct_norm_z_mol_vect_zref_allcomponents)
  H2theta3p_in_degrees = np.degrees(H2theta3p_in_radians)

  H2crossp_norm_z_mol_vect_z_ref_x = np.subtract(np.multiply(H2norm_z_mol_vect_y, z_ref[2]), np.multiply(H2norm_z_mol_vect_z, z_ref[1]))
  H2crossp_norm_z_mol_vect_z_ref_y = np.subtract(np.multiply(H2norm_z_mol_vect_z, z_ref[0]), np.multiply(H2norm_z_mol_vect_x, z_ref[2]))
  H2crossp_norm_z_mol_vect_z_ref_z = np.subtract(np.multiply(H2norm_z_mol_vect_x, z_ref[1]), np.multiply(H2norm_z_mol_vect_y, z_ref[0]))

  H2dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_xcomponent = np.multiply(H2crossp_norm_z_mol_vect_z_ref_x, h2_moved_x)
  H2dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_ycomponent = np.multiply(H2crossp_norm_z_mol_vect_z_ref_y, h2_moved_y)
  H2dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_zcomponent = np.multiply(H2crossp_norm_z_mol_vect_z_ref_z, h2_moved_z)
  H2dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_xycomponents = np.add(H2dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_xcomponent, H2dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_ycomponent)
  H2dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_allcomponents = np.add(H2dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_xycomponents, H2dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_zcomponent)

  H2updated_theta3p_in_radians = np.zeros_like(H2theta3p_in_radians)

  if H2dotproduct_crossp_norm_z_mol_vect_z_ref_h1_moved_allcomponents < 0.0:
      H2updated_theta3p_in_radians = H2theta3p_in_radians / 2.0
  else:
      H2updated_theta3p_in_radians = H2theta3p_in_radians / -2.0

  H2updated_theta3p_in_degrees = np.degrees(H2updated_theta3p_in_radians)
  H2updated_theta3p_in_degrees_all.append(H2updated_theta3p_in_degrees.tolist())

  # ###############################################################################

  #  Defining second quaternion based on axis of rotation and angle for h2 (H2q2)


  H2q2_0 = np.cos(H2updated_theta3p_in_radians)
  H2q2_1 = np.multiply(x_ref[0], np.sin(H2updated_theta3p_in_radians))
  H2q2_2 = np.multiply(x_ref[1], np.sin(H2updated_theta3p_in_radians))
  H2q2_3 = np.multiply(x_ref[2], np.sin(H2updated_theta3p_in_radians))
  ###############################################################################

  #  Calculating the product quaternion for h2.
  e_0 = np.subtract(np.subtract(np.subtract(np.multiply(H2q1_0, H2q2_0), np.multiply(H2q1_1, H2q2_1)), np.multiply(H2q1_2, H2q2_2)), np.multiply(H2q1_3, H2q2_3))
  e_1 = np.subtract(np.add(np.add(np.multiply(H2q1_0, H2q2_1), np.multiply(H2q1_1, H2q2_0)), np.multiply(H2q1_2, H2q2_3)), np.multiply(H2q1_3, H2q2_2))
  e_2 = np.add(np.add(np.subtract(np.multiply(H2q1_0, H2q2_2), np.multiply(H2q1_1, H2q2_3)), np.multiply(H2q1_2, H2q2_0)), np.multiply(H2q1_3, H2q2_1))
  e_3 = np.add(np.subtract(np.add(np.multiply(H2q1_0, H2q2_3), np.multiply(H2q1_1, H2q2_2)), np.multiply(H2q1_2, H2q2_1)), np.multiply(H2q1_3, H2q2_0))
  ##########################################################################

  # H2final_q_0 = copy.deepcopy(e_0)
  # H2final_q_1 = copy.deepcopy(e_1)
  # H2final_q_2 = copy.deepcopy(e_2)
  # H2final_q_3 = copy.deepcopy(e_3)

  H2final_q_0_all.append(e_0)
  H2final_q_1_all.append(e_1)
  H2final_q_2_all.append(e_2)
  H2final_q_3_all.append(e_3)


##############################
# from IPython.display import clear_output  # to show in console the progress

# Initialy the list to create a list of list with size N
N = len(H1final_q_0_all)
distance_matrix = np.zeros((N,N))

for i in range(0, N):
  # clear_output(wait=True)  # to show in console the progress
  for j in range(i+1, N):
    if j != i:

      distance_matrix[i][j] = min (2 * min(((H1final_q_0_all[i] - H1final_q_0_all[j]) ** 2) + ((H1final_q_1_all[i] - H1final_q_1_all[j]) ** 2) + ((H1final_q_2_all[i] - H1final_q_2_all[j]) ** 2) + ((H1final_q_3_all[i] - H1final_q_3_all[j]) ** 2),
                                            ((H1final_q_0_all[i] + H1final_q_0_all[j]) ** 2) + ((H1final_q_1_all[i] + H1final_q_1_all[j]) ** 2) + ((H1final_q_2_all[i] + H1final_q_2_all[j]) ** 2) + ((H1final_q_3_all[i] + H1final_q_3_all[j]) ** 2)),
                                    2 * min(((H1final_q_0_all[i] - H2final_q_0_all[j]) ** 2) + ((H1final_q_1_all[i] - H2final_q_1_all[j]) ** 2) + ((H1final_q_2_all[i] - H2final_q_2_all[j]) ** 2) + ((H1final_q_3_all[i] - H2final_q_3_all[j]) ** 2),
                                            ((H1final_q_0_all[i] + H2final_q_0_all[j]) ** 2) + ((H1final_q_1_all[i] + H2final_q_1_all[j]) ** 2) + ((H1final_q_2_all[i] + H2final_q_2_all[j]) ** 2) + ((H1final_q_3_all[i] + H2final_q_3_all[j]) ** 2)),
                                    2 * min(((H2final_q_0_all[i] - H1final_q_0_all[j]) ** 2) + ((H2final_q_1_all[i] - H1final_q_1_all[j]) ** 2) + ((H2final_q_2_all[i] - H1final_q_2_all[j]) ** 2) + ((H2final_q_3_all[i] - H1final_q_3_all[j]) ** 2),
                                            ((H2final_q_0_all[i] + H1final_q_0_all[j]) ** 2) + ((H2final_q_1_all[i] + H1final_q_1_all[j]) ** 2) + ((H2final_q_2_all[i] + H1final_q_2_all[j]) ** 2) + ((H2final_q_3_all[i] + H1final_q_3_all[j]) ** 2)),
                                    2 * min(((H2final_q_0_all[i] - H2final_q_0_all[j]) ** 2) + ((H2final_q_1_all[i] - H2final_q_1_all[j]) ** 2) + ((H2final_q_2_all[i] - H2final_q_2_all[j]) ** 2) + ((H2final_q_3_all[i] - H2final_q_3_all[j]) ** 2),
                                            ((H2final_q_0_all[i] + H2final_q_0_all[j]) ** 2) + ((H2final_q_1_all[i] + H2final_q_1_all[j]) ** 2) + ((H2final_q_2_all[i] + H2final_q_2_all[j]) ** 2) + ((H2final_q_3_all[i] + H2final_q_3_all[j]) ** 2)))

  # print("Current progress:", np.round(i / len(H1final_q_0_all) * 100, 2), "%")  # to show in console the progress


distance_matrix = distance_matrix + distance_matrix.T - np.diag(np.diag(distance_matrix))
##########################################################################

h1x = new_h1_x.reshape(-1,1)
h1y = new_h1_y.reshape(-1,1)
h1z = new_h1_z.reshape(-1,1)
h2x = new_h2_x.reshape(-1,1)
h2y = new_h2_y.reshape(-1,1)
h2z = new_h2_z.reshape(-1,1)

data_h1 = np.hstack((h1x, h1y, h1z))
data_h2 = np.hstack((h2x, h2y, h2z))
data_all = np.hstack((data_h1, data_h2))
#########################################################################

clusterer = hdbscan.HDBSCAN(metric='precomputed',  min_cluster_size = 50, min_samples = 1, cluster_selection_epsilon=1)
# clusterer = HDBSCAN(metric='precomputed', store_centers = 'medoid',  min_cluster_size = 50, cluster_selection_epsilon = 1, min_samples = 1)
clusterer.fit(distance_matrix)
cluster_count = len(np.unique(clusterer.labels_)) - 1


indx_all_noise = np.where(clusterer.labels_ == -1)

print (f'Hydration Site: {a}')
print (f'# of molecules: {len(clusterer.labels_)}')
print ('cluster count: ', cluster_count)
print (f'# of noises = ',len(indx_all_noise[0]))
print ('persistance = ', clusterer.cluster_persistence_)


#################################################################
for i in range(cluster_count):
  which_water = []
  indx_all = np.where(clusterer.labels_ == i)
  print (f'# of atoms in label {i} = ',len(indx_all[0]))
#   for j in indx_all:
#     which_water.append(data_all[j])

#   outfile = open("/content/drive/MyDrive/Kurtzman/Quaternion_Clustering_HDBscan/Q_" + input_file.rsplit('/')[-1][9:-4] + f'_Label_{i}.xyz', "w")
#   length = len(which_water[0])
#   for k in range (0,length):
#     outfile.write('3\n\n')
#     outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("OW", 0.000,0.000,0.000))
#     outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("H1", which_water[0][k][0],which_water[0][k][1],which_water[0][k][2]))
#     outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("H2", which_water[0][k][3],which_water[0][k][4],which_water[0][k][5]))

#   outfile.close()
# print('----------------------------------\n')

Hydration Site: 19
# of molecules: 8816
cluster count:  2
# of noises =  1469
persistance =  [0.12601067 0.08368304]
# of atoms in label 0 =  2421
# of atoms in label 1 =  4926


In [ ]:
# Centroid for label 0

indx_all = np.where(clusterer.labels_ == 1)

distance_matrix_array = np.array(distance_matrix)
length = len(distance_matrix_array)
which_water = []

# print (f'# of atoms in label = ',len(indx_all[0]))
min_all = []
for j in indx_all[0]:
  distance_for_i = []
  for i2 in range(length):
    if i2 != j:
      distance_for_i.append(distance_matrix_array[i2][j])

  sum = np.sum(distance_for_i)
  min_all.append(sum)
min_distance = np.min(min_all)
print ('min_distance', min_distance)
centroid_where = np.where(min_all == min_distance)
indice_for_indx_all = centroid_where[0] #the indice of our water in indx_all array
centroid = indx_all[0][indice_for_indx_all]
print ('Centroid: ', centroid)

outfile = open("/content/drive/MyDrive/Kurtzman/Quaternion_Clustering_HDBscan/Centroid_" + input_file.rsplit('/')[-1][9:-4] + '_Label_1.xyz', "w")
outfile.write('3\n\n')
outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("OW", 0.000,0.000,0.000))
outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("H1", data_all[centroid][0][0],data_all[centroid][0][1],data_all[centroid][0][2]))
outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("H2", data_all[centroid][0][3],data_all[centroid][0][4],data_all[centroid][0][5]))

outfile.close()


print('----------------------------------\n')


min_distance 3670.270094361689
Centroid:  [5881]
----------------------------------



In [ ]:
# writing the xyz file:

indx_all_noise = np.where(clusterer.labels_ == -1)

print (f'Hydration Site: {a}')
print (f'# of molecules: {len(clusterer.labels_)}')
print ('cluster count: ', cluster_count)
print (f'# of noises = ',len(indx_all_noise[0]))
print ('persistance = ', clusterer.cluster_persistence_)

#################################################################
for i in range(cluster_count):
  which_water = []
  indx_all = np.where(clusterer.labels_ == i)
  print (f'# of atoms in label {i} = ',len(indx_all[0]))
  for j in indx_all:
    which_water.append(data_all[j])

  outfile = open("/content/drive/MyDrive/Kurtzman/Quaternion_Clustering_HDBscan/Q_" + input_file.rsplit('/')[-1][9:-4] + f'_Label_{i}.xyz', "w")
  length = len(which_water[0])
  for k in range (0,length):
    outfile.write('3\n\n')
    outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("OW", 0.000,0.000,0.000))
    outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("H1", which_water[0][k][0],which_water[0][k][1],which_water[0][k][2]))
    outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("H2", which_water[0][k][3],which_water[0][k][4],which_water[0][k][5]))

  outfile.close()
print('----------------------------------\n')


# The rest is extra



In [ ]:
# Cluster_count = No. of clusters(2)
# range (cluster_count) = 0, 1 : (n-1)
# Indx_all = the indices of water molecule (xyz,xyz) that belongs to cluster i (shape = (1,k)) where k is the number of waters in cluster i
# indx_all[0] = the indices array for each cluster
# np.shape(data_all) = (m, 6) where m is the total number of waters
# data_all[j] = the j row (water molecule with j indice)
# which_water = all waters that belong to cluster i (all waters with indices in indx_all), shape = (1, k, 6)
# np.shape(which_water[0]) = (k, 6)

# min_samples = the minimum number of neighbours to a core point.
# The higher this is, the more points are going to be discarded as noise/outliers.

# min_cluster_size = the minimum size a final cluster can be.
# The minimum number of samples in a group for that group to be considered a cluster;
# groupings smaller than this size will be left as noise.
# The higher this is, the bigger your clusters will be.


In [ ]:
# playing with parameters
clusterer = hdbscan.HDBSCAN(metric='precomputed',  min_cluster_size = 100, min_samples = 1)
clusterer.fit(distance_matrix)
cluster_count = len(np.unique(clusterer.labels_)) - 1

##############################################

indx_all_noise = np.where(clusterer.labels_ == -1)
print (f'Hydration Site: {a}')
print (f'# of molecules: {len(clusterer.labels_)}')
print ('cluster count: ', cluster_count)
print (f'# of noises = ',len(indx_all_noise[0]))
print ('persistance = ', clusterer.cluster_persistence_)


for i in range(cluster_count):
  which_water = []
  h1_all = []
  h2_all = []
  indx_all = np.where(clusterer.labels_ == i)
  n = len(indx_all[0])
  print (f'# of atoms in label {i} = ',n)
  for j in indx_all:
    which_water.append(data_all[j])
  length = len(which_water[0])

Hydration Site: 19
# of molecules: 8816
cluster count:  2
# of noises =  1469
persistance =  [0.17516059 0.30510561]
# of atoms in label 0 =  2421
# of atoms in label 1 =  4926


In [ ]:
# writing the xyz file and Centroid together:

indx_all_noise = np.where(clusterer.labels_ == -1)

print (f'Hydration Site: {a}')
print (f'# of molecules: {len(clusterer.labels_)}')
print ('cluster count: ', cluster_count)
print (f'# of noises = ',len(indx_all_noise[0]))
print ('persistance = ', clusterer.cluster_persistence_)
#########################################################


distance_matrix_array = np.array(distance_matrix)
length = len(distance_matrix_array)

for x in range(cluster_count):
  which_water = []
  indx_all = np.where(clusterer.labels_ == x)
  print (f'# of atoms in label {x} = ',len(indx_all[0]))

  for j in indx_all:
    which_water.append(data_all[j])

  outfile = open("/content/drive/MyDrive/Kurtzman/Quaternion_Clustering_HDBscan/Q_" + input_file.rsplit('/')[-1][9:-4] + f'_Label_{x}.xyz', "w")
  length = len(which_water[0])
  for k in range (0,length):
    outfile.write('3\n\n')
    outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("OW", 0.000,0.000,0.000))
    outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("H1", which_water[0][k][0],which_water[0][k][1],which_water[0][k][2]))
    outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("H2", which_water[0][k][3],which_water[0][k][4],which_water[0][k][5]))

  outfile.close()
  min_all = []
  for j in indx_all[0]:
    distance_for_i = []
    for i2 in range(length):
      if i2 != j:
        distance_for_i.append(distance_matrix_array[i2][j])

    sum = np.sum(distance_for_i)
    min_all.append(sum)
  min_distance = np.min(min_all)
  print ('min_distance', min_distance)
  centroid_where = np.where(min_all == min_distance)
  indice_for_indx_all = centroid_where[0]
  print(indice_for_indx_all) #this prints the indice in indx_all array.
  print ('-----')
  centroid = indx_all[0][indice_for_indx_all]
  print (centroid)

  outfile = open("/content/drive/MyDrive/Kurtzman/Quaternion_Clustering_HDBscan/Centroid_" + input_file.rsplit('/')[-1][9:-4] + f'_Label_{x}.xyz', "w")
  length = len(which_water[0])
  outfile.write('3\n\n')
  outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("OW", 0.000,0.000,0.000))
  outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("H1", data_all[centroid][0][0],data_all[centroid][0][1],data_all[centroid][0][2]))
  outfile.write('{:2s}{:9.3f}{:9.3f}{:9.3f}\n'.format("H2", data_all[centroid][0][3],data_all[centroid][0][4],data_all[centroid][0][5]))
  outfile.close()


print('----------------------------------\n')


In [ ]:
clusterer.condensed_tree_
clusterer.single_linkage_tree_.get_clusters(0.023, min_cluster_size=2)